<a href="https://colab.research.google.com/github/Anjasfedo/eceg-lsb-lzw-huffman/blob/main/bit_encoder.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
278 % 255 + 1

24

In [ ]:
# Example string
original_string = "Hello, world! 🌍"

# Encoding the string into UTF-8
encoded_string = original_string.encode('utf-8')
print(f"Encoded (UTF-8) bytes: {encoded_string}")

# Decoding the UTF-8 bytes back into the string
decoded_string = encoded_string.decode('utf-8')
print(f"Decoded string: {decoded_string}")


Encoded (UTF-8) bytes: b'Hello, world! \xf0\x9f\x8c\x8d'
Decoded string: Hello, world! 🌍


In [ ]:
# Convert the string into its UTF-8 encoded byte representation
original_string = "Hello, world! 🌍"

# Encoding the string into UTF-8
encoded_string = original_string.encode('utf-8')

# Convert the encoded bytes to their bit representations
bit_representation = ' '.join(format(byte, '08b') for byte in original_string)
bit_representation


ValueError: Unknown format code 'b' for object of type 'str'

In [ ]:
def encode_with_metadata(message):
    # Step 1: Get the length of the message in bits
    message_bits = ''.join(format(ord(c), '08b') for c in message)
    message_length = len(message_bits)

    # Step 2: Create the metadata (first 32 bits to store the length)
    length_bits = format(message_length, '032b')  # 32-bit binary representation of length

    # Step 3: Concatenate length metadata and message
    full_message = length_bits + message_bits

    return full_message

def decode_with_metadata(bits):
    # Step 1: Extract the first 32 bits for the metadata (length of the message)
    length_bits = bits[:32]
    message_length = int(length_bits, 2)

    # Step 2: Extract the actual message based on the length
    message_bits = bits[32:32 + message_length]

    # Step 3: Convert the message bits back to characters
    message = ''.join(chr(int(message_bits[i:i+8], 2)) for i in range(0, len(message_bits), 8))

    return message

# Example usage:
message = "Hello, World!"
encoded_bits = encode_with_metadata(message)
print(f"Encoded bits: {encoded_bits}")

decoded_message = decode_with_metadata(encoded_bits)
print(f"Decoded message: {decoded_message}")


Encoded bits: 0000000000000000000000000110100001001000011001010110110001101100011011110010110000100000010101110110111101110010011011000110010000100001
Decoded message: Hello, World!


In [ ]:
def encode_with_metadata(bits):
    # Step 1: Get the length of the bit string
    bit_length = len(bits)

    # Step 2: Create the metadata (first 32 bits to store the length)
    length_bits = format(bit_length, '032b')  # 32-bit binary representation of length

    # Step 3: Concatenate length metadata and the actual bit string
    full_message = length_bits + bits

    return full_message

def decode_with_metadata(bits):
    # Step 1: Extract the first 32 bits for the metadata (length of the message)
    length_bits = bits[:32]
    message_length = int(length_bits, 2)

    # Step 2: Extract the actual message based on the length
    message_bits = bits[32:32 + message_length]

    return message_bits

# Example usage:
# Input: a string of bits (no conversion from text)
bits = "0100010101010101010101000100101010101010100010111110100101"
print(f"Original bits: {bits}")

# Encode the bit string with the embedded metadata
encoded_bits = encode_with_metadata(bits)
print(f"Encoded bits with metadata: {encoded_bits}")

# Decode the bit string with the embedded metadata
decoded_bits = decode_with_metadata(encoded_bits)
print(f"Decoded bits: {decoded_bits}")

assert bits == decoded_bits

Original bits: 0100010101010101010101000100101010101010100010111110100101
Encoded bits with metadata: 000000000000000000000000001110100100010101010101010101000100101010101010100010111110100101
Decoded bits: 0100010101010101010101000100101010101010100010111110100101


In [ ]:
def encode_with_padding_info(bits):
    # Step 1: Calculate the padding needed to make the bit length a multiple of 8
    padding_needed = (8 - len(bits) % 8) % 8  # Number of bits to pad

    # Step 2: Embed the padding length in the first 8 bits
    padding_length_bits = format(padding_needed, '08b')  # 8-bit binary representation of padding length

    # Step 3: Pad the message with zeroes (if any padding is needed)
    padded_bits = bits + '0' * padding_needed

    # Step 4: Combine the padding information and the padded bits
    full_message = padding_length_bits + padded_bits

    # Step 5: Convert the full message into 8-bit chunks and encode as ASCII
    encoded_chars = [chr(int(full_message[i:i+8], 2)) for i in range(0, len(full_message), 8)]

    return ''.join(encoded_chars)

def decode_with_padding_info(encoded):
    # Step 1: Convert the encoded ASCII string back to a bit string
    bits = ''.join(format(ord(c), '08b') for c in encoded)

    # Step 2: Extract the first 8 bits as padding length
    padding_length_bits = bits[:8]
    padding_length = int(padding_length_bits, 2)

    # Step 3: Extract the actual message bits (without the padding)
    message_bits = bits[8:]  # Exclude the first 8 bits (padding length)

    # Step 4: Remove the padding bits
    message_bits = message_bits[:-padding_length] if padding_length > 0 else message_bits

    return message_bits

# Example usage:
# Input: a string of bits (no conversion from text)
bits = "0100010101010101010101000100101010101010100010111110100101"
print(f"Original bits: {bits}")

# Encode the bit string with the embedded padding information
encoded_bits = encode_with_padding_info(bits)
print(f"Encoded bits as ASCII: {encoded_bits}")

# Decode the encoded string to get the original bit string back (without padding)
decoded_bits = decode_with_padding_info(encoded_bits)
print(f"Decoded bits: {decoded_bits}")

assert all(1 <= ord(c) <= 255 for c in encoded_bits), "Error: Non-ASCII-safe characters found!"
assert bits == decoded_bits

Original bits: 0100010101010101010101000100101010101010100010111110100101
Encoded bits as ASCII: EUTJªé@
Decoded bits: 0100010101010101010101000100101010101010100010111110100101
